#**Pre-request**

##Mount google drive


In [1]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Install pakages


In [2]:
#Install pakages
%pip install -q -r /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/requirements.txt --no-cache-dir
#import os; os._exit(0)   #  restart automatically

######Restart sesstion after this line at first run to make sure that the new libs are used====

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 246.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 246.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 252.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 224.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 317.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 163.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.9/587.9 kB 355.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 228.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 268.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 134.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
%cd $project_path
%ls /content/drive/MyDrive/Sem-6/coding/github/fraud_detection

/content/drive/MyDrive/Sem-6/coding/github/fraud_detection
clustring_requirements-lock.txt  requirements-lock.txt
clustring_requirements.txt       requirements.txt
configs/                         results/
dataset/                         run_experiment.py
Extract_requirements-lock.txt    sample_extract_requirements-lock.txt
Extract_requirements.txt         src/
notebooks/                       tests/
README.md


##Import  libs

In [13]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import os
import pandas as pd
import yaml
import logging
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
import datetime
import warnings
import re
import sweetviz as sv
import datetime
import datetime
import os


import altair as alt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from google.colab import data_table
data_table.enable_dataframe_formatter()

# ignore warning related to font
warnings.filterwarnings(
    "ignore",
    message="Glyph .* missing from font",  # Pass the pattern as a STRING
    category=UserWarning
)
## Navigate to project location

%pip freeze > requirements-lock.txt



#Basic Methods

##Loging

In [14]:

# Make sure results directory exists
os.makedirs("results", exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("results/data_prep.log")
    ]
)
logger = logging.getLogger(__name__)



##Config

In [15]:
def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file"""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    logger.info(f"Loaded config from {config_path}")
    return config

## CDR dataset

In [16]:
def load_cdr(file_path, nrows=None):
    """Load a CDR CSV file with optional row limit (sample mode)."""
    logger.info(f"Loading file: {file_path} with nrows={nrows}")
    return pd.read_csv(file_path, nrows=nrows)


def load_all_data(config):
    """
    Load all CSVs defined in config into a dict of DataFrames.
    Uses training.sample_size if available.
    """
    base = config["dataset"]["base_path"]
    files = config["dataset"]["files"]
    sample_size = config.get("training", {}).get("sample_size", None)

    data = {}
    for name, fname in files.items():
        path = os.path.join(base, fname)
        df = load_cdr(path, nrows=sample_size)
        data[name] = df
        logger.info(f"Loaded {name} -> {df.shape} from {path}")
    return data


#EDA

##Profling Methods

###ydata

In [17]:

def ydata_generate_profile(df, output_file="results/profile.html", mode="minimal", overwrite=True):
    """
    Generate profiling report (ydata-profiling) with safe defaults.
    """
    # Drop very high-cardinality or long-text columns in minimal mode
    if mode == "minimal":
       drop_cols = []
       for col in df.columns:
        if df[col].dtype == "object":  # only apply to categorical/text
            if df[col].nunique() > 5000 or df[col].astype(str).str.len().mean() > 100:
                drop_cols.append(col)
        elif df[col].nunique() <= 1:  # constant numeric (useless)
            drop_cols.append(col)

    if drop_cols:
        print(f"Skipping columns for profiling: {drop_cols}")
        df = df.drop(columns=drop_cols)


    # Handle overwrite manually
    if overwrite and os.path.exists(output_file):
        os.remove(output_file)

    profile = ProfileReport(
        df,
        title="Fraud Detection EDA Report",
        explorative=True,
        plot={"wordcloud": False},
        correlations={
            "pearson": {"calculate": mode == "full"},
            "spearman": {"calculate": mode == "full"},
            "kendall": {"calculate": mode == "full"},
            "phi_k": {"calculate": mode == "full"},
            "cramers": {"calculate": mode == "full"},
            "auto": {"calculate": mode == "full"},
        },
        interactions={"continuous": mode == "full"},
        missing_diagrams={"heatmap": mode == "full"},
        duplicates={"calculate": mode == "full"}
    )

    profile.to_file(output_file)  # no overwrite argument
    print(f"Profiling report saved → {output_file}")
    return output_file


###sweetviz

In [18]:
def generate_profile_sweetviz(df, output_file="results/profile_sweetviz.html", target=None):
    """
    Generate a Sweetviz report for df (with optional target column).
    """
    drop_cols = []
    for col in df.columns:
        if df[col].dtype == "object":  # only check high-cardinality categoricals
            if df[col].nunique() > 1000:
                drop_cols.append(col)
        elif df[col].dtype == "datetime64[ns]":  # datetime may break plots
            drop_cols.append(col)

    if drop_cols:
        print(f" Dropping columns for Sweetviz: {drop_cols}")
        df = df.drop(columns=drop_cols)

    report = sv.analyze(df, target_feat=target)
    report.show_html(output_file)
    print(f" Sweetviz report generated at {output_file}")
    return output_file


#Preprocessing Functions

##App

In [19]:
def preprocess_app(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the APP dataset and prepare for unified timeline merging.
    """
    # --- Basic cleaning ---
    if "busi_name" in df.columns:
        df["busi_name"] = df["busi_name"].fillna("Unknown").astype(str)

    if "flow" in df.columns:
        df["flow"] = df["flow"].fillna(0)
        df["flow"] = df["flow"].clip(lower=0)
        df["flow_norm"] = df["flow"].apply(lambda x: np.log1p(x))

    # --- Month/time processing ---
    if "month_id" in df.columns:
        df["month_id"] = pd.to_datetime(df["month_id"], errors="coerce", format="%Y-%m")

    if "phone_no_m" in df.columns:
        df["month_id"] = df.groupby("phone_no_m")["month_id"].transform(lambda x: x.ffill().bfill())

    if df["month_id"].isna().sum() > 0:
        median_month = df["month_id"].dropna().median()
        df["month_id"] = df["month_id"].fillna(median_month)

    df["month_str"] = df["month_id"].dt.strftime("%Y-%m")

    # --- 🔹 Timeline preparation (moved from build_unified_timeline) ---
    df["event_time"] = df["month_id"] + pd.offsets.MonthEnd(0)
    df["source"] = "APP"

    # Keep only valid entries
    df = df.dropna(subset=["event_time"])

    return df[["phone_no_m", "event_time", "source"] + [col for col in df.columns if col not in ["phone_no_m", "event_time", "source"]]]


##sms

In [20]:
def preprocess_sms(df):
    if "calltype_id" in df.columns:
      df["calltype_id"] = df["calltype_id"].astype("category")
    return df


##user

In [21]:
def preprocess_user(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the ARPU/User dataset.
    Converts wide monthly ARPU columns (e.g., arpu_201908, arpu_201909, ...)
    into long transactional format suitable for unified timeline integration.
    Retains static user attributes (e.g., city, county, idcard_cnt, label).
    """
    # --- Step 1: Basic cleanup ---
    df["city_name"] = df["city_name"].fillna("Unknown")
    df["county_name"] = df["county_name"].fillna("Unknown")

    if "idcard_cnt" in df.columns and df["idcard_cnt"].isna().sum() > 0:
        df["idcard_cnt"] = df["idcard_cnt"].fillna(df["idcard_cnt"].median())

    # --- Step 2: Detect ARPU columns ---
    arpu_cols = [col for col in df.columns if col.startswith("arpu_")]
    if not arpu_cols:
        raise ValueError("No ARPU columns found (expected columns starting with 'arpu_').")

    # --- Step 3: Melt ARPU columns into long format ---
    id_vars = [c for c in df.columns if c not in arpu_cols]
    df_long = df.melt(
        id_vars=id_vars,
        value_vars=arpu_cols,
        var_name="month_col",
        value_name="arpu_value"
    )

    # --- Step 4: Extract month_id ---
    df_long["month_id"] = (
        df_long["month_col"].str.replace("arpu_", "", regex=False)
        .apply(lambda x: pd.to_datetime(x, format="%Y%m", errors="coerce"))
    )

    # --- Step 5: Clean and normalize ---
    df_long["arpu_value"] = df_long["arpu_value"].fillna(0)
    df_long["arpu_value"] = df_long["arpu_value"].clip(lower=0)

    if "label" in df_long.columns:
        df_long = df_long.dropna(subset=["label"])

    # --- Step 6: Build timeline fields ---
    df_long["event_time"] = df_long["month_id"] + pd.offsets.MonthEnd(0)
    df_long["source"] = "ARPU"

    # --- Step 7: Reorder columns for consistency ---
    ordered_cols = [
        "phone_no_m", "event_time", "source", "month_id",
        "arpu_value", "city_name", "county_name", "idcard_cnt"
    ]
    if "label" in df_long.columns:
        ordered_cols.append("label")

    # Include any remaining columns that might exist but weren’t listed
    remaining_cols = [c for c in df_long.columns if c not in ordered_cols]
    df_long = df_long[ordered_cols + remaining_cols]

    # Drop rows without valid time
    df_long = df_long.dropna(subset=["event_time"])
    df_long = df_long[
        df_long["arpu_value"].notna() & (df_long["arpu_value"] > 0)
    ].reset_index(drop=True)
    return df_long


##Voice

In [22]:
def preprocess_voc(df):
    # Fill categorical columns with 'Unknown' only if missing values exist
    if df["opposite_no_m"].isna().sum() > 0:
        df["opposite_no_m"] = df["opposite_no_m"].fillna("Unknown")
    if df["city_name"].isna().sum() > 0:
        df["city_name"] = df["city_name"].fillna("Unknown")
    if df["county_name"].isna().sum() > 0:
        df["county_name"] = df["county_name"].fillna("Unknown")
    if df["imei_m"].isna().sum() > 0:
        df["imei_m"] = df["imei_m"].fillna("Unknown")

    # calltype_id → EDA shows no missing, so don’t overwrite unnecessarily

    # Drop rows missing datetime
    if df["start_datetime"].isna().sum() > 0:
        df = df.dropna(subset=["start_datetime"])

    # Handle call duration
    if df["call_dur"].isna().sum() > 0:
        df["call_dur"] = df["call_dur"].fillna(0)



    return df


#Load data

In [23]:
config = load_config("configs/baseline.yaml")

# Load raw data
%ls /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/raw/sample/sampled/
data = load_all_data(config)




sample_train_app.csv  sample_train_user.csv
sample_train_sms.csv  sample_train_voc.csv



# Excute EDA
# ===============================


In [24]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
for name, df in data.items():

    # -------------------------------
    # Load dataset-specific preprocessing config
    # -------------------------------
    preprocess_cfg = config["preprocessing"].get(name, {})

    numeric_features = preprocess_cfg.get("numeric_features", [])
    categorical_features = preprocess_cfg.get("categorical_features", [])
    datetime_features = preprocess_cfg.get("datetime_features", [])
    target = preprocess_cfg.get("target", None)

    print(f"\nPreprocessing {name}")
    print(f"   numeric: {numeric_features}")
    print(f"   categorical: {categorical_features}")
    print(f"   datetime: {datetime_features}")
    print(f"   target: {target}")

    # -------------------------------
    # Profiling with ydata_profiling
    # -------------------------------
    profiling_mode = config["training"].get("profiling_mode", "minimal")
    ydata_output = f"results/profile_{name}_{timestamp}.html"
    ydata_generate_profile(df, output_file=ydata_output, mode=profiling_mode)

    # -------------------------------
    # Profiling with Sweetviz
    # -------------------------------
    sweetviz_output = f"results/profile_{name}_sweetviz_{timestamp}.html"
    generate_profile_sweetviz(df, output_file=sweetviz_output, target=target)




Preprocessing app
   numeric: ['flow', 'month_id']
   categorical: ['busi_name']
   datetime: []
   target: None
Skipping columns for profiling: ['phone_no_m']


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00,  6.48it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

...

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Profiling report saved → results/profile_voc_20251011_125048.html
 Dropping columns for Sweetviz: ['opposite_no_m', 'start_datetime']


                                             |          | [  0%]   00:00 -> (? left)

Report results/profile_voc_sweetviz_20251011_125048.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
 Sweetviz report generated at results/profile_voc_sweetviz_20251011_125048.html


#Preprocessing

In [25]:


timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
for name, df in data.items():
  if name == "app":
      print(f"\nPreprocessing {name}")
      df = preprocess_app(df)
      df.to_csv(f"dataset/CallChinses/preprocessed/preprocessed_{name}_{timestamp}.csv", index=False)
      display(df.head())
  elif name == "sms":
      print(f"\nPreprocessing {name}")
      df = preprocess_sms(df)
      df.to_csv(f"dataset/CallChinses/preprocessed/preprocessed_{name}_{timestamp}.csv", index=False)
      display(df.head())
  elif name == "user":
    print(f"\nPreprocessing {name}")
    df = preprocess_user(df)
    df.to_csv(f"dataset/CallChinses/preprocessed/preprocessed_{name}_{timestamp}.csv", index=False)
    display(df.head())
  elif name == "voc":
    print(f"\nPreprocessing {name}")
    df = preprocess_voc(df)
    df.to_csv(f"dataset/CallChinses/preprocessed/preprocessed_{name}_{timestamp}.csv", index=False)
    display(df.head())




Preprocessing app


,phone_no_m,event_time,source,busi_name,flow,month_id,flow_norm,month_str
0,9855c1ff2c8b1e8c5bff0c0675ee1902cae41b5ad47f61...,2019-11-30,APP,Unknown,0.000000,2019-11-01,0.000000,2019-11
1,9b2ba2cf84a3aa4dc2ea40c0143744e4213e12ae036258...,2019-11-30,APP,Unknown,0.009801,2019-11-01,0.009753,2019-11
2,9b2ba2cf84a3aa4dc2ea40c0143744e4213e12ae036258...,2019-10-31,APP,微信,0.036454,2019-10-01,0.035805,2019-10
3,9b2ba2cf84a3aa4dc2ea40c0143744e4213e12ae036258...,2019-10-31,APP,QingMang,0.081688,2019-10-01,0.078523,2019-10
4,9b2ba2cf84a3aa4dc2ea40c0143744e4213e12ae036258...,2019-11-30,APP,百度地图,0.014647,2019-11-01,0.014540,2019-11



Preprocessing sms


,phone_no_m,opposite_no_m,calltype_id,request_datetime
0,e96ba2776ede5a56fffed9c86483f87725ab0d2cb7591c...,9ad756cda2a8927e40831746c5f98bb756661b05b96e02...,2,2019-08-02 16:00:50
1,d0c57e1d2099b0abc33b61cc5e8a3454e96c363b5888df...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-14 17:12:22
2,d0c57e1d2099b0abc33b61cc5e8a3454e96c363b5888df...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-14 17:12:22
3,d0c57e1d2099b0abc33b61cc5e8a3454e96c363b5888df...,659d611d78c760cb6495bdc0fe9d0430ee565816687858...,2,2019-08-14 19:48:31
4,d0c57e1d2099b0abc33b61cc5e8a3454e96c363b5888df...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-14 17:52:01



Preprocessing user


,phone_no_m,event_time,source,month_id,arpu_value,city_name,county_name,idcard_cnt,label,month_col
0,f35f40086eb60a0966ac441b60ce5862f43acb43871397...,2019-08-31,ARPU,2019-08-01,49.1,Unknown,Unknown,1,1,arpu_201908
1,ac2eb79b2020d27b76031ba4b7ef4a243d1d1363f0c1c1...,2019-08-31,ARPU,2019-08-01,1.0,广安,广安区分公司,1,1,arpu_201908
2,b8370fac19ea042d8bf5c213c604eb57e3a6e13ed4a8e3...,2019-08-31,ARPU,2019-08-01,104.2,德阳,广汉分公司,4,1,arpu_201908
3,43748d52a6f7c109f5a733bde6ffa2092b7fb8414e997c...,2019-08-31,ARPU,2019-08-01,2.0,广安,广安区分公司,0,1,arpu_201908
4,5ea1171f9227762f179a2e57ee5c0a1f5e6b546e70b245...,2019-08-31,ARPU,2019-08-01,109.8,德阳,中江分公司,3,1,arpu_201908



Preprocessing voc


,phone_no_m,opposite_no_m,calltype_id,start_datetime,call_dur,city_name,county_name,imei_m
0,4b16c23a42fdf352951e035f80b0e0b5d0c168cb1cfd90...,d7fc4a5a58ace035962dda80b204ea96ff33691cd10d70...,1,2020-03-20 08:34:56,57,Unknown,Unknown,4dee9441de400bc581d4e6ea19fd07c3185c804145567d...
1,4b16c23a42fdf352951e035f80b0e0b5d0c168cb1cfd90...,b5d946533247cec13ec443e088275a4e4c525b1db93cb8...,1,2020-03-20 10:09:26,280,Unknown,Unknown,4dee9441de400bc581d4e6ea19fd07c3185c804145567d...
2,4b16c23a42fdf352951e035f80b0e0b5d0c168cb1cfd90...,f07ce48bb9914e07e4cae3d7fcbf8d204b124d51a65275...,1,2020-03-20 10:05:05,45,Unknown,Unknown,4dee9441de400bc581d4e6ea19fd07c3185c804145567d...
3,4b16c23a42fdf352951e035f80b0e0b5d0c168cb1cfd90...,d7fc4a5a58ace035962dda80b204ea96ff33691cd10d70...,1,2020-03-20 09:16:46,47,Unknown,Unknown,4dee9441de400bc581d4e6ea19fd07c3185c804145567d...
4,4b16c23a42fdf352951e035f80b0e0b5d0c168cb1cfd90...,fdcbd8d6c8d8cfb9849aaa2f0de040a44e0f54d2aad837...,1,2020-03-20 08:30:04,43,Unknown,Unknown,4dee9441de400bc581d4e6ea19fd07c3185c804145567d...
